In [1]:
import pandas as pd
from pandas.core.common import flatten
import importlib
import pandas as pd
import osmnx
import numpy as np
import matplotlib.pyplot as plt
import pymongo
from shapely.geometry import LineString,Polygon,Point
import shapely

In [2]:
myclient = pymongo.MongoClient("mongodb://192.168.120.127:27017")
mydb = myclient["tmap2"]
mylink = mydb["link1"]
myway = mydb["osm_edge"]
mylink_update = mydb["link2"]

In [3]:
districts = [
    {'city':'杭州市','district':'上城区'},
    {'city':'杭州市','district':'下城区'},
    {'city':'杭州市','district':'拱墅区'},
    {'city':'杭州市','district':'西湖区'},
    {'city':'杭州市','district':'江干区'},
    {'city':'杭州市','district':'滨江区'},
]

In [4]:
def get_district_gdf(districts):
    district_docs = []
    for district in districts:
        doc = mydb.district.find_one(district)
        district_docs.append(doc)
    district_gdf = pd.DataFrame(district_docs)
    return district_gdf

In [38]:
district_gdf = get_district_gdf(districts)

In [39]:
district_gdf

,_id,boundingbox,city,class,display_name,district,geojson,icon,importance,lat,licence,lon,osm_id,osm_type,place_id,province,q,type
0,199044257,"[30.1949431, 30.2607296, 120.1323896, 120.2096...",杭州市,boundary,"Shangcheng District, Hangzhou City, Zhejiang, PRC",上城区,"{'type': 'Polygon', 'coordinates': [[[120.1323...",https://nominatim.openstreetmap.org/images/map...,0.399088,30.22795245,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.180160810303,4591409,relation,199044257,浙江省,"上城区,杭州市,浙江省,中国",administrative
1,248409958,"[30.2601237, 30.3533804, 120.1477521, 120.2076...",杭州市,boundary,"Xiacheng District, Hangzhou City, Zhejiang, PRC",下城区,"{'type': 'Polygon', 'coordinates': [[[120.1477...",https://nominatim.openstreetmap.org/images/map...,0.399263,30.3064869,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.172492156078,4591385,relation,248409958,浙江省,"下城区,杭州市,浙江省,中国",administrative
2,248409957,"[30.2750087, 30.4025918, 120.0805322, 120.212199]",杭州市,boundary,"Gongshu District, Hangzhou City, Zhejiang, PRC",拱墅区,"{'type': 'Polygon', 'coordinates': [[[120.0805...",https://nominatim.openstreetmap.org/images/map...,0.399961,30.33882115,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.132549671251,4591384,relation,248409957,浙江省,"拱墅区,杭州市,浙江省,中国",administrative
3,198746762,"[30.0777045, 30.357607, 119.9930893, 120.1761091]",杭州市,boundary,"Xihu District, Hangzhou City, Zhejiang, PRC",西湖区,"{'type': 'Polygon', 'coordinates': [[[119.9930...",https://nominatim.openstreetmap.org/images/map...,0.315521,30.21733695,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.077743417214,4591410,relation,198746762,浙江省,"西湖区,杭州市,浙江省,中国",administrative
4,198835215,"[30.2322421, 30.3945626, 120.179497, 120.4092819]",杭州市,boundary,"Jianggan District, Hangzhou City, Zhejiang, PRC",江干区,"{'type': 'Polygon', 'coordinates': [[[120.1794...",https://nominatim.openstreetmap.org/images/map...,0.396764,30.313707,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.29578582157,4591400,relation,198835215,浙江省,"江干区,杭州市,浙江省,中国",administrative
5,199241411,"[30.1350798, 30.2385559, 120.1183624, 120.2324...",杭州市,boundary,"Binjiang District, Hangzhou City, Zhejiang, PRC",滨江区,"{'type': 'Polygon', 'coordinates': [[[120.1183...",https://nominatim.openstreetmap.org/images/map...,0.399961,30.1865188,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.17602888358,3221179,relation,199241411,浙江省,"滨江区,杭州市,浙江省,中国",administrative


In [5]:
import geopandas as gpd

In [43]:
district_gdf = gpd.GeoDataFrame(district_gdf)
district_gdf['polygon'] = district_gdf['geojson'].apply(shapely.geometry.shape)
district_gdf = district_gdf.set_geometry('polygon',crs='EPSG:4326')
district_gdf

,_id,boundingbox,city,class,display_name,district,geojson,icon,importance,lat,licence,lon,osm_id,osm_type,place_id,province,q,type,polygon
0,199044257,"[30.1949431, 30.2607296, 120.1323896, 120.2096...",杭州市,boundary,"Shangcheng District, Hangzhou City, Zhejiang, PRC",上城区,"{'type': 'Polygon', 'coordinates': [[[120.1323...",https://nominatim.openstreetmap.org/images/map...,0.399088,30.22795245,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.180160810303,4591409,relation,199044257,浙江省,"上城区,杭州市,浙江省,中国",administrative,"POLYGON ((120.1323896 30.199699, 120.1347642 3..."
1,248409958,"[30.2601237, 30.3533804, 120.1477521, 120.2076...",杭州市,boundary,"Xiacheng District, Hangzhou City, Zhejiang, PRC",下城区,"{'type': 'Polygon', 'coordinates': [[[120.1477...",https://nominatim.openstreetmap.org/images/map...,0.399263,30.3064869,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.172492156078,4591385,relation,248409958,浙江省,"下城区,杭州市,浙江省,中国",administrative,"POLYGON ((120.1477521 30.291688, 120.1479836 3..."
2,248409957,"[30.2750087, 30.4025918, 120.0805322, 120.212199]",杭州市,boundary,"Gongshu District, Hangzhou City, Zhejiang, PRC",拱墅区,"{'type': 'Polygon', 'coordinates': [[[120.0805...",https://nominatim.openstreetmap.org/images/map...,0.399961,30.33882115,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.132549671251,4591384,relation,248409957,浙江省,"拱墅区,杭州市,浙江省,中国",administrative,"POLYGON ((120.0805322 30.3336899, 120.082152 3..."
3,198746762,"[30.0777045, 30.357607, 119.9930893, 120.1761091]",杭州市,boundary,"Xihu District, Hangzhou City, Zhejiang, PRC",西湖区,"{'type': 'Polygon', 'coordinates': [[[119.9930...",https://nominatim.openstreetmap.org/images/map...,0.315521,30.21733695,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.077743417214,4591410,relation,198746762,浙江省,"西湖区,杭州市,浙江省,中国",administrative,"POLYGON ((119.9930893 30.1867131, 119.9932545 ..."
4,198835215,"[30.2322421, 30.3945626, 120.179497, 120.4092819]",杭州市,boundary,"Jianggan District, Hangzhou City, Zhejiang, PRC",江干区,"{'type': 'Polygon', 'coordinates': [[[120.1794...",https://nominatim.openstreetmap.org/images/map...,0.396764,30.313707,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.29578582157,4591400,relation,198835215,浙江省,"江干区,杭州市,浙江省,中国",administrative,"POLYGON ((120.179497 30.2855536, 120.1795775 3..."
5,199241411,"[30.1350798, 30.2385559, 120.1183624, 120.2324...",杭州市,boundary,"Binjiang District, Hangzhou City, Zhejiang, PRC",滨江区,"{'type': 'Polygon', 'coordinates': [[[120.1183...",https://nominatim.openstreetmap.org/images/map...,0.399961,30.1865188,"Data © OpenStreetMap contributors, ODbL 1.0. h...",120.17602888358,3221179,relation,199241411,浙江省,"滨江区,杭州市,浙江省,中国",administrative,"POLYGON ((120.1183624 30.176089, 120.1185553 3..."


In [44]:
polygon_unary = district_gdf.polygon.unary_union

In [45]:
osm_json = osmnx.osm_net_download(polygon=polygon_unary,network_type='drive')

In [46]:
def osm_node_way_df_from_response_jsons(response_jsons):
    nodes = {}
    paths = {}
    for osm_data in response_jsons:
        nodes_temp, paths_temp = osmnx.parse_osm_nodes_paths(osm_data)
        for key, value in nodes_temp.items():
            nodes[key] = value
        for key, value in paths_temp.items():
            paths[key] = value
    nodes_df = pd.DataFrame.from_dict(nodes, orient='index')
    ways_df = pd.DataFrame.from_dict(paths, orient='index')
    return nodes_df,ways_df

In [47]:
osm_node_df,osm_way_df = osm_node_way_df_from_response_jsons(osm_json)

In [48]:
osm_node_df.head()

,osmid,x,y,highway,ref
25926073,25926073,120.165330,30.279548,traffic_signals,NaN
25926074,25926074,120.166437,30.280177,NaN,NaN
25926076,25926076,120.167065,30.280508,NaN,NaN
25926077,25926077,120.168054,30.280856,NaN,NaN
25926079,25926079,120.169065,30.280983,NaN,NaN


In [50]:
import geojson
def calc_node_geometry_geojson(noderow):
    node_geometry_geojson = geojson.Point((noderow['x'],noderow['y']))
    return node_geometry_geojson
osm_node_df['geometry_geojson'] = osm_node_df.apply(calc_node_geometry_geojson,axis=1)
osm_node_df.head()

,osmid,x,y,highway,ref,geometry_geojson
25926073,25926073,120.165330,30.279548,traffic_signals,NaN,"{'type': 'Point', 'coordinates': [120.1653304,..."
25926074,25926074,120.166437,30.280177,NaN,NaN,"{'type': 'Point', 'coordinates': [120.1664368,..."
25926076,25926076,120.167065,30.280508,NaN,NaN,"{'type': 'Point', 'coordinates': [120.1670653,..."
25926077,25926077,120.168054,30.280856,NaN,NaN,"{'type': 'Point', 'coordinates': [120.1680539,..."
25926079,25926079,120.169065,30.280983,NaN,NaN,"{'type': 'Point', 'coordinates': [120.1690646,..."


In [28]:
osm_way_df.head()

,osmid,highway,nodes,name,tunnel,maxspeed,oneway,bridge,ref,lanes,access,width,junction,geometry_geojson
4296592,4296592,tertiary,"[25954988, 25954989, 25954990, 5500598897, 550...",朝晖路,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.16..."
4298707,4298707,secondary,"[26197735, 5585650835, 26197734, 26414460, 348...",体育场路,NaN,NaN,yes,NaN,NaN,2,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.15..."
4298708,4298708,primary,"[26198714, 262909253]",大关路,NaN,NaN,yes,NaN,NaN,2,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.15..."
4305243,4305243,secondary,"[26041229, 26391714, 268978266, 25926086]",建国北路,NaN,NaN,yes,NaN,NaN,2,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.17..."
4305248,4305248,primary,"[26041466, 26683076]",环城北路,NaN,NaN,yes,NaN,NaN,2,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.18..."


In [52]:
def calc_way_geometry_geojson(nodes):
    def get_node_xy(node_id):
        node = osm_node_df.loc[node_id]
        return node['x'],node['y']
    way_geometry_geojson = geojson.LineString([get_node_xy(node_id) for node_id in nodes])
    return way_geometry_geojson
osm_way_df['geometry_geojson'] = osm_way_df['nodes'].apply(calc_way_geometry_geojson)

In [53]:
osm_way_df.head()

,osmid,nodes,highway,name,maxspeed,tunnel,oneway,bridge,lanes,ref,access,width,junction,geometry_geojson
4296592,4296592,"[25954988, 25954989, 25954990, 5500598897, 550...",tertiary,朝晖路,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.16..."
4298707,4298707,"[26197735, 5585650835, 26197734, 26414460, 348...",secondary,体育场路,NaN,NaN,yes,NaN,2,NaN,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.15..."
4298708,4298708,"[26198714, 262909253]",primary,大关路,NaN,NaN,yes,NaN,2,NaN,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.15..."
4305243,4305243,"[26041229, 26391714, 268978266, 25926086]",secondary,建国北路,NaN,NaN,yes,NaN,2,NaN,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.17..."
4305248,4305248,"[26041466, 26683076]",primary,环城北路,NaN,NaN,yes,NaN,2,NaN,NaN,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.18..."


In [58]:
osm_way_df.iloc[1]['geometry_geojson']['coordinates'][0]

[120.1502435, 30.2730562]

In [30]:
osm_graph = osmnx.core.create_graph(osm_json, name='unnamed', retain_all=False, network_type='drive')
simple_graph = osmnx.simplify.simplify_graph(osm_graph)
undirect_simple_graph = osmnx.save_load.get_undirected(simple_graph)

In [31]:
osm_graph_node,osm_graph_edge = osmnx.save_load.graph_to_gdfs(osm_graph, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [32]:
simpe_graph_node,simple_graph_edge = osmnx.save_load.graph_to_gdfs(simple_graph, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [33]:
simple_graph_edge.head()

,access,bridge,geometry,highway,junction,key,lanes,length,maxspeed,name,oneway,osmid,ref,tunnel,u,v,width
0,NaN,NaN,"LINESTRING (120.0586011 30.2389999, 120.058507...",tertiary,NaN,0,NaN,633.995,NaN,杨梅山路,False,553954463,NaN,NaN,322043913,3601760665,NaN
1,NaN,NaN,"LINESTRING (120.0586011 30.2389999, 120.059172...",unclassified,NaN,0,NaN,58.294,NaN,NaN,False,29285059,NaN,NaN,322043913,322044026,NaN
2,NaN,NaN,"LINESTRING (120.0586011 30.2389999, 120.058620...",tertiary,NaN,0,NaN,376.965,NaN,杨梅山路,False,553954463,NaN,NaN,322043913,322043886,NaN
3,NaN,NaN,"LINESTRING (120.1218066 30.276413, 120.1218212...",residential,NaN,0,NaN,53.759,NaN,工专路,False,559362889,NaN,NaN,5395185688,320120027,NaN
4,NaN,NaN,"LINESTRING (120.1218066 30.276413, 120.1210778...",tertiary,NaN,0,NaN,206.030,NaN,黄姑山横路,False,514946124,NaN,NaN,5395185688,26705484,NaN


In [34]:
simple_graph_edge.loc[0]

access                                                    NaN
bridge                                                    NaN
geometry    LINESTRING (120.0586011 30.2389999, 120.058507...
highway                                              tertiary
junction                                                  NaN
key                                                         0
lanes                                                     NaN
length                                                633.995
maxspeed                                                  NaN
name                                                     杨梅山路
oneway                                                  False
osmid                                               553954463
ref                                                       NaN
tunnel                                                    NaN
u                                                   322043913
v                                                  3601760665
width   

In [35]:
type(simple_graph_edge)

geopandas.geodataframe.GeoDataFrame

In [6]:
link_df = gpd.GeoDataFrame(list(mylink.find()))

In [27]:
pd.DataFrame(link_df.iloc[1])

,1
_added_direction,False
_id,5c2d02fa31f04af0c4d1a7d6
_name,杨梅山路
access,NaN
bridge,NaN
geometry_json,"{'type': 'LineString', 'coordinates': [[120.05..."
highway,tertiary
id1,vP8yxWlTZekMYRsBrAOVb
id2,Dnp7VbeflAe8P3fjVKLr0SwpZ0J0uDYAO53HBY8Qy2uKy6...
junction,NaN


In [9]:
def gen_osm_id(*args):
    hashid = hashids.encode(*args)
    return hashid

In [62]:
import geopy.distance
def get_distance(origin_point, new_point):
    dist = geopy.distance.distance(tuple(origin_point), tuple(new_point)).m
    return dist

In [60]:
def divid_edge_row(row):
    nodes = row['nodes']
    name = row['name']
    # the edge name should be single and nodes should not be closed
    if type(name) == str and len(nodes) == len(set(nodes)):
        if name in way_dict.keys():
            array_temp = nodes[1:-1]
            nodes_to_bedivid = [nodes[0]]
            for i in range(len(array_temp)):
                if array_temp[i] in way_dict[name]:
                    nodes_to_bedivid.append(array_temp[i])
            nodes_to_bedivid.append(nodes[-1])
            edge_divided = pd.DataFrame()
            if len(nodes_to_bedivid) > 2:
                for j in range(len(nodes_to_bedivid) - 1):
                    link_df_rowcp = row.copy()
                    link_df_rowcp['u'], link_df_rowcp['v'] = nodes_to_bedivid[j], nodes_to_bedivid[j + 1]
                    link_df_rowcp['id1'] = gen_osm_id(*link_df_rowcp['osmid_tuple'], link_df_rowcp['u'],
                                                      link_df_rowcp['v'])
                    index1, index2 = nodes.index(nodes_to_bedivid[j]), nodes.index(nodes_to_bedivid[j + 1])
                    link_df_rowcp['nodes'] = nodes[index1:(index2 + 1)]
                    link_df_rowcp['geometry_json'] = calc_way_geometry_geojson(link_df_rowcp['nodes'])
                    node_front = link_df_rowcp['geometry_json']['coordinates'][0][::-1]
                    node_end = link_df_rowcp['geometry_json']['coordinates'][-1][::-1]
                    link_df_rowcp['length'] = get_distance(node_front, node_end)
                    link_df_rowcp['id2'] = gen_osm_id(*link_df_rowcp['nodes'])
                    link_df_rowcp = pd.DataFrame(link_df_rowcp).T
                    edge_divided = pd.concat([edge_divided, link_df_rowcp])
                    # edge_divided.append(link_df_rowcp, ignore_index=True)
                return edge_divided
        return pd.DataFrame(row).T
    return pd.DataFrame(row).T

In [11]:
def dump_json(path):
    myway_dict = {}
    for x in myway.find():
        name = x["name"]
        if str(name)!="nan":
            if name not in myway_dict.keys():
                myway_dict[name] = [x["u"],x["v"]]
            else:
                nodes_to_process = [x["u"],x["v"]]
                for item in nodes_to_process:
                    if item not in myway_dict[name]:
                        myway_dict[name].append(item)
    with open(path,'w') as f:
        json.dump(myway_dict,f)

def read_json(path):
    with open(path,'r') as f:
        data = json.load(f)
    return data

In [13]:
% cd ..
import json
json_path = 'data/name.json'
dump_json(json_path)
way_dict = read_json(json_path)

/media/sunmengnan/F_Disk/TR/gitlab/edge-subdivision


In [14]:
import hashids
from hashids import Hashids

In [15]:
hashids = Hashids(salt='this is my salt 1')

In [63]:
processed = divid_edge_row(link_df.loc[1])

In [66]:
processed

,_added_direction,_id,_name,access,bridge,geometry_json,highway,id1,id2,junction,...,name,nodes,oneway,osmid,osmid_tuple,ref,tunnel,u,v,width
1,False,5c2d02fa31f04af0c4d1a7d6,杨梅山路,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.04...",tertiary,YyjkvPWT7ypDQZFvD2Z4e,4EAOWwbcVbDQpeHpolWJncJB93Q3IQgZ93JFjr6JDkHBwL...,NaN,...,杨梅山路,"[3601760662, 5346586480, 5346586481, 534658648...",False,553954463,[553954463],NaN,NaN,3601760662,322043913,NaN
1,False,5c2d02fa31f04af0c4d1a7d6,杨梅山路,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",tertiary,vP8yxWlTZekMYRsgJP3DA,b9dDnYRfw89eBJ,NaN,...,杨梅山路,"[322043913, 5346586516]",False,553954463,[553954463],NaN,NaN,322043913,5346586516,NaN
1,False,5c2d02fa31f04af0c4d1a7d6,杨梅山路,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",tertiary,8Qyrl7jFo0jD8zf6218jB,g0kYVDEtnb1ekMiOEWQnOCYgp5ejC68OyR4tgwDyodTbj7...,NaN,...,杨梅山路,"[5346586516, 5180292950, 5180292951, 518029295...",False,553954463,[553954463],NaN,NaN,5346586516,2652716479,NaN


In [102]:
link_df_update = link_df_update.drop(['_id'], axis=1)

In [90]:
link_df_update = gpd.GeoDataFrame(link_df_update)

In [ ]:
link_df_update = pd.DataFrame()

for index, row in link_df.iterrows():
    row_divid = divid_edge_row(row)
    link_df_update = pd.concat([link_df_update, divid_edge_row(row)], ignore_index=True)

In [91]:
link_df_update_query = link_df_update[link_df_update['name']=='鲤鱼山路']['length']
link_df_update_query

17895    148.525
17896    145.103
17897    355.025
17899    148.525
17900    145.103
17901    355.025
17902    148.525
17903    145.103
17904    355.025
17905    330.708
43067    148.525
43068    145.103
43069    355.025
43070    330.708
Name: length, dtype: object

In [92]:
link_df_update.index.size

48482

In [103]:
link_df_update.head()

,_added_direction,_name,access,bridge,geometry_json,highway,id1,id2,junction,key,...,name,nodes,oneway,osmid,osmid_tuple,ref,tunnel,u,v,width
0,False,,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",unclassified,40DgV7c7kWj5nsex9JKo,wxv3PnVczlZd1qUo7wgv9ueX6zjWI6yM9qDu803bpzclOx...,NaN,0,...,,"[322043913, 322044026, 322044027, 322044028, 3...",False,29285059,[29285059],NaN,NaN,322043913,322044026,NaN
1,False,杨梅山路,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.04...",tertiary,YyjkvPWT7ypDQZFvD2Z4e,4EAOWwbcVbDQpeHpolWJncJB93Q3IQgZ93JFjr6JDkHBwL...,NaN,0,...,杨梅山路,"[3601760662, 5346586480, 5346586481, 534658648...",False,553954463,[553954463],NaN,NaN,3601760662,322043913,NaN
2,False,杨梅山路,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",tertiary,vP8yxWlTZekMYRsgJP3DA,b9dDnYRfw89eBJ,NaN,0,...,杨梅山路,"[322043913, 5346586516]",False,553954463,[553954463],NaN,NaN,322043913,5346586516,NaN
3,False,杨梅山路,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.05...",tertiary,8Qyrl7jFo0jD8zf6218jB,g0kYVDEtnb1ekMiOEWQnOCYgp5ejC68OyR4tgwDyodTbj7...,NaN,0,...,杨梅山路,"[5346586516, 5180292950, 5180292951, 518029295...",False,553954463,[553954463],NaN,NaN,5346586516,2652716479,NaN
4,True,杨梅山路,NaN,NaN,"{'type': 'LineString', 'coordinates': [[120.04...",tertiary,YyjkvPWT7ypDQZFvD2Z4e,4EAOWwbcVbDQpeHpolWJncJB93Q3IQgZ93JFjr6JDkHBwL...,NaN,0,...,杨梅山路,"[3601760662, 5346586480, 5346586481, 534658648...",False,553954463,[553954463],NaN,NaN,3601760662,322043913,NaN


In [104]:
mydb.link2.delete_many({})
mydb.link2.insert_many(link_df_update.to_dict('records'))
